<img src="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png" srcset="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_130 130w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_260 260w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_390 390w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_520 520w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_650 650w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_780 780w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_910 910w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1040 1040w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1170 1170w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1290 1290w" sizes="100vw" width="1290">

# Advanced Tool Calling with LangChain

This lab demonstrates sophisticated tool calling with LangChain using a room booking system scenario. You'll learn how to implement type-safe tools with proper annotations and compose multiple tools into a cohesive system.

Key concepts covered:
- Type-annotated tool implementations
- Custom business logic
- LangChain agent integration

# Install required packages

In [ ]:
!pip -q install langchain==0.3.7 langchain-openai==0.2.9 python-dateutil==2.8.2 python-dotenv

### Read environment variables from .env

In [ ]:
%load_ext dotenv
%dotenv

import os

## Tools Implementation

We'll create four tools that work together:
1. User lookup - Get user details by username
2. Room listing - Find available rooms by capacity/features
3. Room booking - Book rooms with conflict checking
4. Booking history - View user's bookings

In [ ]:
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union
from dateutil.parser import parse
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.agents.agent import RunnableMultiActionAgent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Simulated database of users and bookings
USER_DB = {
    "john.doe": {"id": "U123", "name": "John Doe", "department": "Engineering"},
    "jane.smith": {"id": "U456", "name": "Jane Smith", "department": "Marketing"}
}

ROOM_DB = {
    "A101": {"capacity": 4, "features": ["whiteboard", "projector"]},
    "B202": {"capacity": 8, "features": ["whiteboard", "video-conf"]},
    "C303": {"capacity": 12, "features": ["whiteboard", "projector", "video-conf"]}
}

BOOKINGS_DB: List[Dict] = []  # Stores active bookings

In [ ]:
@tool
def get_user_info(username: str) -> Union[Dict[str, str], str]:
    """Retrieve user information from the system.

    Args:
        username (str): The username to look up (e.g., 'john.doe')

    Returns:
        Dict[str, str]: User information including ID, name, and department

    Raises:
        ValueError: If the username is not found
    """
    if username not in USER_DB:
        return f"User '{username}' not found"

    return USER_DB[username]

@tool
def list_available_rooms(capacity: Optional[int] = None, features: Optional[List[str]] = None) -> Union[List[str], str]:
    """List available meeting rooms matching the criteria.

    Args:
        capacity (Optional[int]): Minimum required capacity
        features (Optional[List[str]]): Required features (e.g., ['whiteboard', 'projector'])

    Returns:
        List[str]: List of room IDs matching the criteria
    """
    available_rooms = []

    for room_id, room_info in ROOM_DB.items():
        if capacity and room_info['capacity'] < capacity:
            continue

        if features:
            if not all(feature in room_info['features'] for feature in features):
                continue

        available_rooms.append(room_id)

    return available_rooms

@tool
def book_room(
    user_id: str,
    room_id: str,
    start_time: str,
    duration_minutes: int
) -> Union[Dict[str, Union[str, datetime]], str]:
    """Book a meeting room for a specified time period.

    Args:
        user_id (str): ID of the user making the booking
        room_id (str): ID of the room to book
        start_time (str): Start time in ISO format (YYYY-MM-DD HH:MM)
        duration_minutes (int): Duration of booking in minutes

    Returns:
        Dict[str, Union[str, datetime]]: Booking confirmation with details

    Raises:
        ValueError: If the room is unavailable or inputs are invalid
    """
    # Validate room exists
    if room_id not in ROOM_DB:
        return f"Room '{room_id}' does not exist"

    # Parse and validate start time
    try:
        start_datetime = parse(start_time)
    except ValueError:
        return "Invalid start time format. Use YYYY-MM-DD HH:MM"

    # Validate duration
    if duration_minutes <= 0 or duration_minutes > 240:  # Max 4 hours
        return "Duration must be between 1 and 240 minutes"

    end_datetime = start_datetime + timedelta(minutes=duration_minutes)

    # Check for conflicts
    for booking in BOOKINGS_DB:
        if booking['room_id'] == room_id:
            booking_start = booking['start_time']
            booking_end = booking['end_time']

            if (start_datetime < booking_end and end_datetime > booking_start):
                return f"Room {room_id} is already booked during this time"

    # Create booking
    booking = {
        'booking_id': f'BK{len(BOOKINGS_DB)+1:03d}',
        'user_id': user_id,
        'room_id': room_id,
        'start_time': start_datetime,
        'end_time': end_datetime
    }

    BOOKINGS_DB.append(booking)

    return booking

@tool
def get_user_bookings(user_id: str) -> List[Dict]:
    """Retrieve all bookings for a specific user.

    Args:
        user_id (str): ID of the user to look up bookings for

    Returns:
        List[Dict]: List of booking details for the user
    """
    return [booking for booking in BOOKINGS_DB if booking['user_id'] == user_id]

## Tools "Agent" Setup

Now we'll create a LangChain tools "agent" that can use our tools to handle natural language booking requests.

In [ ]:
# Initialize the LLM and create the agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)

from langchain_community.tools import HumanInputRun

human_tool = HumanInputRun()
tools = [human_tool, get_user_info, list_available_rooms, book_room, get_user_bookings]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that manages room bookings. Use the available tools to help users book rooms and manage their reservations."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

if isinstance(agent_executor.agent, RunnableMultiActionAgent):
    agent_executor.agent.stream_runnable = False

## Example Usage

Let's test our booking system with a natural language request.

In [ ]:
# Example usage
query = "I am jane.smith, and I need to book a room for 6 people" # with a projector tomorrow at 3pm for 1 hour"

result = agent_executor.invoke({"input": query})
print(result["output"])